In [1]:
# import json

# with open('all_qguide_data.html', 'r') as file:
#     json_data = json.load(file)

# with open('all_qguide_data.json', 'w') as file:
#     json.dump(json_data, file, separators=(',', ':'))


In [8]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from functools import lru_cache
from tabulate import tabulate

@lru_cache(maxsize=None)
def extract_table_data(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
    except requests.RequestException:
        return None

    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    # Find the feedback header link
    feedback_header = soup.find('a', {'href': lambda x: x and x.startswith('#report')})
    if not feedback_header:
        return None

    # Extract text from the header
    title_text = feedback_header.get('title', '')
    if not title_text:
        title_text = feedback_header.text

    # Extract instructor name and course title using regex
    import re

    # Find the last hyphen before <br>
    match = re.search(r'Feedback for\s+(.*?)\s*-\s*([^-<\n]+?)(?:\s*<br>|\s*\(click)', title_text)

    if not match:
        return None

    full_course_title = match.group(1).strip()
    instructor_name = match.group(2).strip()

    # Handle multiple departments in course title
    departments = []
    course_parts = full_course_title.split(',')
    for part in course_parts:
        dept = part.strip().split()[0]
        if dept not in departments:
            departments.append(dept)

    department = ', '.join(departments)

    # Skip certain tables that we don't want to process
    captions_to_skip = [
        "Table for [QTitle]-In this course, most students listen attentively with an open mind and a willingness to change their point of view as they learn more about the topic.-Statistics.",
        "Table for [QTitle]-In this course (including sections), I feel comfortable expressing my views on controversial topics.-Statistics.",
        "Table for [qtitle]-in this course, most students listen attentively with an open mind and a willingness to change their point of view as they learn more about the topic..",
        "Table for [qtitle]-in this course (including sections), i feel comfortable expressing my views on controversial topics..",
        "Table for [qtitle]-in this course, most students listen attentively with an open mind and a willingness to change their point of view as they learn more about the topic.."
    ]

    captions_to_skip = [caption.lower().strip() for caption in captions_to_skip]

    # Find all tables in the HTML
    tables = soup.find_all('table')
    dataframes = {}

    # Store basic course info
    dataframes['Instructor Name'] = pd.DataFrame([instructor_name], columns=['Instructor Name'])
    dataframes['Course Name'] = pd.DataFrame([full_course_title], columns=['Course Name'])
    dataframes['Department'] = pd.DataFrame([department], columns=['Department'])

    # Process each table
    for table in tables:
        # Find the nearest preceding h3 tag to use as the table name
        h3_tag = table.find_previous('h3')
        if h3_tag:
            table_name = "_".join(h3_tag.text.strip().lower().split()).replace("-", "_")
        else:
            continue

        caption_tag = table.find('caption')
        if caption_tag:
            caption_text = caption_tag.text.strip().lower()
            if any(skip_caption in caption_text for skip_caption in captions_to_skip):
                continue

        # Extract headers and data
        headers = [th.text.strip() for th in table.find('tr').find_all('th')]

        data = []
        for row in table.find_all('tr'):
            row_data = [cell.text.strip() for cell in row.find_all(['td', 'th'])]
            if len(row_data) == len(headers):
                data.append(row_data)

        # Create DataFrame if we have data
        if len(data) > 1:  # Skip if only headers
            df = pd.DataFrame(data[1:], columns=headers)
            dataframes[table_name] = df

    return dataframes

In [9]:
url = "https://harvard.bluera.com/harvard/rpv-eng.aspx?lang=eng&redi=1&SelectedIDforPrint=2a25bbb354dd26d2a5018145b3eccac802438e93cb86651665bfd7dc8f5e8b888dcf958e60eb641327093338906fe195&ReportType=2&regl=en-US"
tables_data = extract_table_data(url)

# Print instructor name and each table with its name
for table_name, table_df in tables_data.items():
    print(table_name, table_df)
    print(tabulate(table_df, headers='keys', tablefmt='grid'))
    print()

Instructor Name   Instructor Name
0   Glenda Carpio
+----+-------------------+
|    | Instructor Name   |
+====+===================+
|  0 | Glenda Carpio     |
+----+-------------------+

Course Name   Course Name
0  AFRAMER 55
+----+---------------+
|    | Course Name   |
+====+===============+
|  0 | AFRAMER 55    |
+----+---------------+

Department   Department
0    AFRAMER
+----+--------------+
|    | Department   |
+====+==============+
|  0 | AFRAMER      |
+----+--------------+

course_response_rate            Raters Students
0       Responded        5
1         Invited        7
2  Response Ratio      71%
+----+----------------+------------+
|    | Raters         | Students   |
+====+================+============+
|  0 | Responded      | 5          |
+----+----------------+------------+
|  1 | Invited        | 7          |
+----+----------------+------------+
|  2 | Response Ratio | 71%        |
+----+----------------+------------+

course_general_questions                     

In [10]:
import json
import time
from concurrent.futures import ThreadPoolExecutor

with open('all_qguide_data.json', 'r') as f:
    all_qguide_data = json.load(f)

course_data_list = []

total_entries = len(all_qguide_data['data'])
processed_entries = 0
start_time = time.time()

def process_entry(entry):
    course_data = {key.capitalize(): entry[key] for key in ['title', 'url', 'instructor', 'department', 'term', 'subject', 'blueCourseId'] if key in entry}
    tables_data = extract_table_data(entry['url'])
    
    # Handle the case where tables_data is None
    if tables_data is None:
        course_data['Feedback'] = {}
    else:
        tables_data_dict = {table_name: table_df.to_dict(orient='records') for table_name, table_df in tables_data.items()}
        course_data['Feedback'] = tables_data_dict

    return course_data

def update_progress():
    global processed_entries
    processed_entries += 1
    percent_finished = (processed_entries / total_entries) * 100
    avg_time_per_iteration = (time.time() - start_time) / processed_entries
    remaining_entries = total_entries - processed_entries
    remaining_time_seconds = remaining_entries * avg_time_per_iteration

    hours = int(remaining_time_seconds // 3600)
    remaining_time_seconds %= 3600
    minutes = int(remaining_time_seconds // 60)
    seconds = int(remaining_time_seconds % 60)

    print(f"Iteration {processed_entries}/{total_entries} - {percent_finished:.2f}% completed - Estimated time remaining: {hours}h {minutes}m {seconds}s")

with ThreadPoolExecutor(max_workers=16) as executor:
    futures = []
    for entry in all_qguide_data['data']:
        futures.append(executor.submit(process_entry, entry))

    for future in futures:
        try:
            result = future.result()
            if result is not None:
                course_data_list.append(result)
            update_progress()
        except Exception as e:
            print(f"Error processing entry: {e}")
            update_progress()

with open('course_data.json', 'w') as json_file:
    json.dump(course_data_list, json_file, indent=4)

Iteration 1/11961 - 0.01% completed - Estimated time remaining: 2h 29m 55s
Iteration 2/11961 - 0.02% completed - Estimated time remaining: 1h 28m 49s
Iteration 3/11961 - 0.03% completed - Estimated time remaining: 0h 59m 12s
Iteration 4/11961 - 0.03% completed - Estimated time remaining: 0h 46m 13s
Iteration 5/11961 - 0.04% completed - Estimated time remaining: 0h 36m 58s
Iteration 6/11961 - 0.05% completed - Estimated time remaining: 0h 30m 49s
Iteration 7/11961 - 0.06% completed - Estimated time remaining: 0h 26m 24s
Iteration 8/11961 - 0.07% completed - Estimated time remaining: 0h 26m 11s
Iteration 9/11961 - 0.08% completed - Estimated time remaining: 0h 23m 16s
Iteration 10/11961 - 0.08% completed - Estimated time remaining: 0h 20m 57s
Iteration 11/11961 - 0.09% completed - Estimated time remaining: 0h 19m 2s
Iteration 12/11961 - 0.10% completed - Estimated time remaining: 0h 17m 27s
Iteration 13/11961 - 0.11% completed - Estimated time remaining: 0h 16m 6s
Iteration 14/11961 - 0.

# This below section is not possible anymore because I forgot to scrape instructor names from 2019 :(

In [65]:
# import json

# def add_unique_courses(json1, json2):
#     # Create a set of Bluecourseid from the first JSON to keep track of added courses
#     existing_course_ids = {course['Bluecourseid'] for course in json1}
    
#     # Loop through the second JSON and add courses that don't exist in the first
#     for course in json2:
#         if course['Bluecourseid'] not in existing_course_ids:
#             json1.append(course)  # Add the new course to the first JSON
    
#     return json1

# # Load JSON data from files
# with open('course_data.json', 'r') as file1:
#     json1 = json.load(file1)

# with open('new_course_data.json', 'r') as file2:
#     json2 = json.load(file2)

# # Add unique courses from json2 to json1
# combined_json = add_unique_courses(json1, json2)

# # Save the combined JSON array
# with open('2020-2024_Course_Data.json', 'w') as outfile:
#     json.dump(combined_json, outfile, indent=4)


In [11]:
import pandas as pd
import json
from tabulate import tabulate

# Load the data
with open('course_data.json', 'r') as f:
    course_data = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(course_data)

# Display the general head of the data to understand the structure
data_sample = df.head(10)
print("Data sample (general head):")
print(tabulate(data_sample, headers='keys', tablefmt='grid'))

Data sample (general head):
+----+-------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+--------------------------------------+-------------+--------------------------------------+-------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
import pandas as pd
import json
import numpy as np

# Load the data
with open('course_data.json', 'r') as f:
    course_data = json.load(f)

# Create a dictionary to store aggregated data per instructor
instructor_data = {}

for course in course_data:
    feedback = course.get('Feedback', {})
    instructor_name = feedback.get('Instructor Name', {})
    if not instructor_name:
        continue

    instructor_name = instructor_name[0].get('Instructor Name')
    if not instructor_name:
        continue

    department = feedback.get('Department', {})
    if department:
        department = department[0].get('Department')
    else:
        department = "Unknown"

    for table_name, records in feedback.items():
        if table_name == "general_instructor_questions":
            if instructor_name not in instructor_data:
                instructor_data[instructor_name] = {
                    'departments': department.split(", "),  # Split multiple departments into a list
                    'excellent_count': 0,
                    'very_good_count': 0,
                    'good_count': 0,
                    'fair_count': 0,
                    'unsatisfactory_count': 0,
                    'total_responses': 0
                }

            course_info = records[0]
            try:
                total_count = float(course_info['Count'])

                def safe_percentage(value):
                    if value == 'N/A':
                        return 0
                    return float(value.strip('%')) / 100

                instructor_data[instructor_name]['excellent_count'] += total_count * safe_percentage(course_info['Excellent'])
                instructor_data[instructor_name]['very_good_count'] += total_count * safe_percentage(course_info['Very Good'])
                instructor_data[instructor_name]['good_count'] += total_count * safe_percentage(course_info['Good'])
                instructor_data[instructor_name]['fair_count'] += total_count * safe_percentage(course_info['Fair'])
                instructor_data[instructor_name]['unsatisfactory_count'] += total_count * safe_percentage(course_info['Unsatisfactory'])
                instructor_data[instructor_name]['total_responses'] += total_count
            except (ValueError, KeyError):
                continue

# Convert to DataFrame
df = pd.DataFrame.from_dict(instructor_data, orient='index')

def calculate_bayesian_rating(row):
    try:
        # Calculate weighted sum for the rating
        weighted_sum = (row['excellent_count'] * 5 +
                        row['very_good_count'] * 4 +
                        row['good_count'] * 3 +
                        row['fair_count'] * 2 +
                        row['unsatisfactory_count'] * 1)

        # Prior parameters (weakly informative prior centered around 4)
        alpha_prior = 4
        beta_prior = 1

        # Update parameters with data
        alpha_posterior = alpha_prior + weighted_sum
        beta_posterior = beta_prior + row['total_responses'] * 5 - weighted_sum

        # Calculate lower bound of 95% credible interval
        lower_bound = alpha_posterior / (alpha_posterior + beta_posterior) * 5

        return lower_bound
    except (ValueError, KeyError):
        return None

# Apply the Bayesian calculation to each row
df['Bayesian Rating'] = df.apply(calculate_bayesian_rating, axis=1)

# Calculate overall percentiles for Bayesian Rating
df['Percentile Rank'] = df['Bayesian Rating'].rank(pct=True) * 100

# Define grade boundaries based on your specification
grade_boundaries = [
    (0, 0.1, 'S+'),
    (0.1, 0.5, 'S'),
    (0.5, 1, 'S-'),
    (1, 2, 'A+'),
    (2, 5, 'A'),
    (5, 10, 'A-'),
    (10, 20, 'B+'),
    (20, 40, 'B'),
    (40, 50, 'B-'),
    (50, 60, 'C+'),
    (60, 70, 'C'),
    (70, 80, 'C-'),
    (80, 90, 'D'),
    (90, 100, 'F'),
]

# Assign letter grades based on percentile rank
def assign_letter_grade(percentile):
    for lower, upper, grade in grade_boundaries:
        if lower <= 100-percentile < upper:
            return grade
    return 'F'  # default if something goes wrong

df['Overall Letter Grade'] = df['Percentile Rank'].apply(assign_letter_grade)

# Calculate department-specific percentile ranks and assign department-specific letter grades
def calculate_department_grades(df):
    department_letter_grades = []

    for index, row in df.iterrows():
        departments = row['departments']
        department_grades = []

        for department in departments:
            # Calculate the department-specific percentile rank
            dept_percentile_rank = df[df['departments'].apply(lambda x: department in x)]['Bayesian Rating'].rank(pct=True) * 100
            dept_grade = assign_letter_grade(dept_percentile_rank.loc[index])
            department_grades.append(dept_grade)

        # Take the lowest grade among the department-specific grades
        if department_grades:
            lowest_grade = min(department_grades, key=lambda grade: grade_boundaries.index(next(b for b in grade_boundaries if b[2] == grade)))
        else:
            lowest_grade = 'F'

        department_letter_grades.append(lowest_grade)

    return pd.Series(department_letter_grades, index=df.index)

df['Department Letter Grade'] = calculate_department_grades(df)

# Create final output DataFrame with just the columns we want
result_df = pd.DataFrame({
    'Instructor Name': df.index,
    'Departments': df['departments'].apply(lambda x: ", ".join(x)),
    'Total Ratings': df['total_responses'],
    'Bayesian Rating': df['Bayesian Rating'],
    'Overall Letter Grade': df['Overall Letter Grade'],
    'Department Letter Grade': df['Department Letter Grade']
}).sort_values(by='Bayesian Rating', ascending=False)

# Display results
print(result_df.to_string(index=False))

# Calculate and print common statistical measurements for the numerical columns
numerical_columns = ['Total Ratings', 'Bayesian Rating']

for column in numerical_columns:
    print(f"\nStatistics for '{column}':")
    print(f"  Mean: {result_df[column].mean():.2f}")
    print(f"  Standard Deviation: {result_df[column].std():.2f}")
    print(f"  Minimum: {result_df[column].min():.2f}")
    print(f"  Maximum: {result_df[column].max():.2f}")
    print(f"  Median: {result_df[column].median():.2f}")
    print(f"  25th Percentile (Q1): {result_df[column].quantile(0.25):.2f}")
    print(f"  75th Percentile (Q3): {result_df[column].quantile(0.75):.2f}")


                       Instructor Name                Departments  Total Ratings  Bayesian Rating Overall Letter Grade Department Letter Grade
                      Katharine Clarke                      EXPOS           84.0         4.988235                   S+                      S+
                      William Friedman                     FRSEMR           47.0         4.979167                   S+                      S+
                              Jing Cai                      CHNSE           83.0         4.976310                   S+                      S+
                       Susan Farbstein                     FRSEMR           39.0         4.975000                    S                      S-
                       Cirrus Foroughi                       ECON           35.0         4.972222                    S                      S+
                      Nicole Newendorp                    SOC-STD           34.0         4.971429                    S                      S+